In [1]:
from bokeh.plotting import figure, show, output_notebook, reset_output
from bokeh.models import ColumnDataSource
from bokeh.layouts import row

from collections import Counter
from tabulate import tabulate
from tqdm import tqdm, tnrange, tqdm_notebook

from nltk.tag import StanfordPOSTagger
from sner import Ner, POSClient

from nltk.corpus import wordnet

from gensim.models import KeyedVectors, Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models.phrases import Phraser, Phrases
from scipy.sparse import dia_matrix
from sklearn import metrics
from sklearn.cluster import AgglomerativeClustering

#import hdbscan
import nltk
import numpy as np
import os
import pandas as pd
import pickle
import sklearn

os.chdir('D:/Scraping')

%run -i "fun.py"

<font size=3>
    
Per alleggerire il codice, le funzioni che sono già state create in altri notebook sono state salvate in un file `.py` che viene caricato insieme alle librerie una volta specificata la directory di riferimento. In questo modo si possono chiamare le funzioni definite dall'utente senza dover riproporre il codice.

<br></br>
# [1. Modello w2v da addestrare](#1)

## [1.1 Creazione modello w2v](#1.1)

<font size=3>
Non avendo ottenuto delle grandi performance con il modello già addestrato si prova a creare un modello basandosi sui dati a disposizione. Il primo step è quindi la lettura del dataframe.

In [2]:
with open('imdb_df4', "rb") as input_file:
    imdb_df = pickle.load(input_file)

<font size=3>
    
I passaggi di pre-processing sono già stati realizzati e si passa direttamente alla creazione di una lista in cui ogni elemento rappresenta una recensione.

In [3]:
all_sentences = []
for review in tqdm(imdb_df["Sentences4"]):
    for sentence in review:
        all_sentences.append(sentence.split(' '))

100%|███████████████████████████████████████████████████████████████████████| 784259/784259 [00:31<00:00, 24760.85it/s]


<font size=3>
    
Diverse espressioni sono la combinazione di due o più parole che prese singolarmente hanno un significato diverso. Identificare queste espressioni e non considerare sempre le parole come termini singoli può aiutrare il modello a creare una miglior rappresentazione dei termini. Per facilitare il compito al modello è possibile creare una lista, `common_terms` che contiene un insieme di termini comuni spesso utilizzati per combinare delle parole. Con `Phrases` si identificano i possibili candidati ad essere considerati delle espressioni e con `Phrases` si selezionano i più frequenti.

Il risultato è un insieme di liste in cui i concetti sono uniti fra di loro con degli underscore per formare un'unica parola.

In [4]:
%%time
common_terms = ["of", "with", "without", "and", "or", "the", "a"]
phrases = Phrases(all_sentences, common_terms=common_terms)
bigram = Phraser(phrases)

all_sentences = list(bigram[all_sentences])

Wall time: 11min 50s


<font size=3>
    
La seguente classe è creata a partire dal modulo `CallbackAny2Vec` importato da `gensim`. La libreria di default non mette a disposizione una funzione o un parametro per visualizzare la qualità della progressione durante la crazione del modello. Definendo questa classe, richiamandola nell'addestramento del modello, è possibile visualizzare la funzione di perdita ad ogni epoca. 

In [5]:
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

<font size=3>
    
La funzione `Word2Vec` permette di creare un nuovo modello dato in input un corpus di documenti. Si inizializza la funzione con i seguenti parametri:
* `min_count`: è la frequenza minima di una parola o espressione per essere considerata nell'output finale. La soglia di $500$ è stata scelta appositamente alta in quanto si è notato dagli altri modelli che la selezione esclusiva dei termini più frequenti migliora le performance classificative.
* `size`: è la dimensione del vettore in output. Più è alto il valore più spazio il modello ha disposizione per creare la rappresentazione del termine, il problema può essere che eccessivo spazio sforzi il modello a dover creare dei valori nel vettore non del tutto pertinenti.
* `workers`: il numero di core da utilizzare, più è alto più il processo è veloce, ma la tempistica non scala in maniera completamente lineare.
* `window`: il numero di parole da considerare in una finestra per predire il termine mancante (la tecnica utilizzata è quella di <b>CBOW</b> in cui dato il contesto si cerca di predire un termine mancante).

Al modello inizializzato vengono passate tutte le frasi ricavate in precedenza. L'ultima funzione è quella di train nel quale il modello viene effettivamente addestrato sui dati e si crea la rappresentazione vettoriale dei termini/espressioni. I parametri in ingresso, oltre al corpus, sono:
* `total_examples`: dimensione del corpus.
* `epochs`: dietro l'architettura del modello c'è una rete neurale a due layer strutturati appositamente per individuare delle proprietà del linguaggio; questo parametro indica il numero di iterazioni per addestrare la rete.
* `compute_loss`: calcola la perdita del modello ad ogni iterazione.
* `callbacks`: mostra a schermo i valori della funzione di perdita ad ogni epoca. 

In [6]:
%%time
model = Word2Vec(min_count=300, size=300, workers=5, window=5)
model.build_vocab(all_sentences)

model.train(all_sentences, total_examples=model.corpus_count, epochs=30, compute_loss=True, callbacks=[callback()])

Loss after epoch 0: 35742944.0
Loss after epoch 1: 23500944.0
Loss after epoch 2: 11247800.0
Loss after epoch 3: 5198264.0
Loss after epoch 4: 5272624.0
Loss after epoch 5: 5283704.0
Loss after epoch 6: 5281440.0
Loss after epoch 7: 5222488.0
Loss after epoch 8: 5168880.0
Loss after epoch 9: 5097008.0
Loss after epoch 10: 5003544.0
Loss after epoch 11: 4916296.0
Loss after epoch 12: 4838296.0
Loss after epoch 13: 4754416.0
Loss after epoch 14: 4637888.0
Loss after epoch 15: 3051192.0
Loss after epoch 16: 0.0
Loss after epoch 17: 0.0
Loss after epoch 18: 0.0
Loss after epoch 19: 0.0
Loss after epoch 20: 0.0
Loss after epoch 21: 0.0
Loss after epoch 22: 0.0
Loss after epoch 23: 0.0
Loss after epoch 24: 0.0
Loss after epoch 25: 0.0
Loss after epoch 26: 0.0
Loss after epoch 27: 0.0
Loss after epoch 28: 0.0
Loss after epoch 29: 0.0
Wall time: 30min 2s


(3537894865, 5138421750)

<font size=3>
    
La ragione per cui la funzione di perdita cala a zero dopo diverse iterazioni è da ricercarsi nel modo in cui la libreria gestisce i valori, come suggerito da [questa](https://stackoverflow.com/questions/59823688/gensim-word2vec-model-loss-becomes-0-after-few-epochs) discussione e [un bug report](https://github.com/RaRe-Technologies/gensim/issues/2735). La funzione di perdita di per sé non restituisce il valore ad ogni epoca ma un valore complessivo che somma la perdita ad ogni iterazione partendo dal primissimo modello. Per ottenere il valore corretto ad ogni epoca è stato necessario creare una funzione in cui si faceva la differenza tra la perdita dello step attuale e quello precedente. In questa sottrazione è possibile che i due fattori siano numeri molto grandi che non possono essere rappresentati nel formato `float32` e quindi i due sottraendi risultano nulli. 

<font size=3>
    
Si salva il modello appena creato.

In [7]:
model.save("IMDB_word2vec.model")

<font size=3>
    
Un altro [bug presente su gensim](https://github.com/RaRe-Technologies/gensim/issues/2136) riguarda la funzione `callbacks()` che è stata applicata per ricavare la funzione di perdita ad ogni epoca. Per via di questa informazione aggiuntiva il modello salvato non può essere caricato normalmente tramite `load` della libreria. 
Non esiste una soluzione universale al problema, in questo caso sembrerebbe che richiamare la classe `callback` definita nel notebook prima del caricamento del modello risolvi la questione.

In [7]:
model = Word2Vec.load("IMDB_word2vec.model")

<br></br>
## [1.2 Matrice di similarità](#1.2)

<font size=3>
    
Si riproducono gli step che portano alla creazione della matrice di similarità, partendo dal loading dei pattern di parole e tag.

In [8]:
with open('w_pattern', "rb") as input_file:
    w_pattern = pickle.load(input_file)
    
with open('t_pattern', "rb") as input_file:
    t_pattern = pickle.load(input_file)

<font size=3>
    
Dall'insieme di pattern si effettua un conteggio della frequenza e si prende lo $0.5\%$ più comune.

In [9]:
# senza spelling correction
di_p1 = {}
for pattern in t_pattern:
    for st in pattern:
        if st not in di_p1.keys():
            di_p1[st] = 1
        else:
            di_p1[st] += 1
            
pat_temp1 = Counter(di_p1).most_common()

# si seleziona lo 0.5% più comune
gsp1 = Counter(di_p1).most_common(int(len(pat_temp1)/200))

# si passa da tupla a dizionario
gsp1_diz = {}
for pattern in gsp1:
    if pattern[0] not in gsp1_diz.keys():
        gsp1_diz[pattern[0]] = 1
        
# si creano due nuove liste di parole e tag relativi solamente ai pattern rilevanti
w_pattern_gsp, t_pattern_gsp = relevant_pattern(gsp1_diz, t_pattern, w_pattern)

100%|██████████████████████████████████████████████████████████████████| 10113411/10113411 [00:19<00:00, 531247.72it/s]


<font size=3>
    
Si crea una lista con i nomi nei pattern rilevanti e solo i nomi compresi all'interno del modello w2v creato.

In [10]:
word_accepted2 = []
nouns_list2 = []
word_no_spell2 = find_word2(w_pattern_gsp, t_pattern_gsp, w_output=word_accepted2, nouns_list=nouns_list2)

  0%|                                                                                     | 0/10113411 [00:00<?, ?it/s]fun.py:50: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  model[temp_w[k]]
100%|██████████████████████████████████████████████████████████████████| 10113411/10113411 [01:34<00:00, 107053.45it/s]


<font size=3>
    
Ognuno di questi termini sarà una riga/colonna della matrice di similarità che viene inizializzata nel codice seguente. Per ragioni di memoria la matrice non è creata direttamente ma si utilizza un array.

In [11]:
print(len(word_accepted2))

m = len(word_accepted2)
distances_w2v_imdb = dia_matrix((m , m)).toarray()

3891


<font size=3>
    
Per ogni coppia di parole si calcola la similarità coseno e il valore viene inserito nella relativa cella. Per velocizzare il processo, poiché la similarità è simmetrica, si creano i valori solo nella matrice diagonale superiore.

In [12]:
cont = 0
for i in tqdm(range(len(word_accepted2))):
    j = i
    while j < len(word_accepted2):
        distances_w2v_imdb[i, j] = model.similarity(word_accepted2[i], word_accepted2[j])
        j = j + 1

  0%|                                                                                         | 0/3891 [00:00<?, ?it/s]fun.py:5: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  for i in tqdm(range(len(pattern_list))):
100%|██████████████████████████████████████████████████████████████████████████████| 3891/3891 [01:57<00:00, 33.08it/s]


<font size=3>
    
Si mettono i valori anche nella parte inferiore girando gli indici di ogni cella, questo passaggio è necessario in quanto l'algoritmo di clustering richiede una matrice completa in input. I valori sono approssimati e infine si effettua il passaggio da array a matrice vera e propria, anche questo richiesto dall'algoritmo di clustering, e si ottiene il complementare ad uno dei valori in modo tale da passare da matrice di similarità a matrice delle distanze, l'unico formato possibile ammesso in input per il clustering.

In [13]:
for i in tqdm(range(distances_w2v_imdb.shape[0])):
    for j in range(distances_w2v_imdb.shape[1]):
        if j < i:
            distances_w2v_imdb[i, j] = distances_w2v_imdb[j, i]


distances_w2v_imdb = distances_w2v_imdb.round(3)

distances_w2v_imdb = np.matrix(distances_w2v_imdb)
d_matrix1 = 1 - distances_w2v_imdb

100%|████████████████████████████████████████████████████████████████████████████| 3891/3891 [00:03<00:00, 1264.80it/s]


<font size=3>
    
Si carica il dizionario ottenuto nell'altro notebook con un elenco di parole e i loro sinonimi.

In [14]:
with open('diz_similarity', "rb") as input_file:
    diz_similarity = pickle.load(input_file)

<font size=3>
    
Per non ripetere tutti gli step del modello già addestrato, compresi quelli che non hanno avuto delle buone performance, la matrice viene ridotta di dimensione togliendo quelle parole che erano state considerate come sinonimi dalla similarità di <b>Wu-Palmer</b> tramite il modello WordNet.

In [15]:
# si trovano i sinonimi
keys_lower = [word.lower() for word in diz_similarity.keys()]
diz_similarity2 = {}
for i in range(len(keys_lower)):
    diz_similarity2[keys_lower[i]] = list(diz_similarity.values())[i]
    
    
# si aggiunge un elemento nullo in corrispondenza dei sinonimi
new_w2v_words = []
filter_w2v = []
for word in word_accepted2:
    if (word in diz_similarity2.keys()) and (word not in filter_w2v):
        new_w2v_words.append(word)
        for w in diz_similarity2[word]:
            filter_w2v.append(w)
    else:
        new_w2v_words.append('')
       
    
# dagli indici si eliminano dalla matrice i relativi elementi
idx_w2v = [i for i, x in enumerate(new_w2v_words) if x == '']
idx_w2v.reverse()

n_matrix_w2v = np.delete(d_matrix1, (idx_w2v), axis=0)
n_matrix_w2v = np.delete(n_matrix_w2v, (idx_w2v), axis=1)

w2v_words = [word for word in new_w2v_words if word != '']

print(n_matrix_w2v.shape)

(3160, 3160)


<br></br>
## [1.3 Clustering](#1.3)

<font size=3>
    
Dalla nuova matrice si realizza una cluster analysis come già fatto per altri modelli. Avendo già notato che una tipologia di link non porta a buoni risultati, si utilizzeranno solamente le altre due e anche il numero di cluster massimi da creare è ridotto per adattarsi alle nuove dimensioni della matrice.

In [16]:
link = ['average', 'complete']
silhouette_score_w2v, CH_score_w2v = [[] for i in range(2)]
(silhouette_w2v, CH_w2v) = clustering(5, 31, link_list=link, dist=n_matrix_w2v, 
                              silhouette_score=silhouette_score_w2v, CH_score=CH_score_w2v)

In [17]:
c = [i for i in range(5, 31)]
h_plot_evaluation(title="Word2vec Silhouette index", metric=silhouette_w2v[0], c=c, label1="Average",
                  metric2=silhouette_w2v[1], label2="Complete", pos_legend="top_right")

<font size=3>
    
Si prende la lista dei mille termini più comuni e solo su questi si realizzerà una nuova cluster analysis.

In [18]:
common_words2 = Counter(nouns_list2).most_common()

new_w2v_words2 = []
filter_w2v2 = []
for word in common_words2[0:1000]:
    if word[0] in diz_similarity.keys():
        for w in diz_similarity[word[0]]:
            filter_w2v2.append(w)

    if word[0] not in filter_w2v2:
        new_w2v_words2.append(word[0])
    else:
        new_w2v_words2.append('')

<font size=3>
    
Rimozione delle parole dalla matrice.

In [19]:
idx_w2v2 = [w2v_words.index(word) for word in w2v_words if word not in new_w2v_words2]
idx_w2v2.reverse()

n_matrix_w2v2 = np.delete(n_matrix_w2v, (idx_w2v2), axis=0)
n_matrix_w2v2 = np.delete(n_matrix_w2v2, (idx_w2v2), axis=1)

print(n_matrix_w2v2.shape)

(831, 831)


In [20]:
silhouette_score_w2v2, CH_score_w2v2 = [[] for i in range(2)]
(silhouette_w2v2, CH_w2v2) = clustering(2, 31, link_list=link, dist=n_matrix_w2v2, 
                                        silhouette_score=silhouette_score_w2v2, CH_score=CH_score_w2v2)

In [21]:
c = [i for i in range(2, 31)]
h_plot_evaluation(title="Word2vec filter Silhouette index", metric=silhouette_w2v2[0], c=c, label1="Average",
                  metric2=silhouette_w2v2[1], label2="Complete", pos_legend="top_left")

In [ ]:
model = AgglomerativeClustering(affinity='precomputed', n_clusters=12, linkage="average").fit(n_matrix1)
labels = model.labels_
Counter(labels).most_common()

In [ ]:
temp = pd.DataFrame({'Word': primary_words, 'Cluster': labels})

temp["Freq"] = temp["Word"].map(dict(common_words))
temp = temp.sort_values(by='Freq', ascending=False)

temp.head()
temp.to_csv(path_or_buf="D:/Scraping/temp_noroot.csv", index=False)

<br></br>
<br></br>
# [2. Clustering con spelling](#2.)

<font size=3>
    
Come preannunciato, il lavoro si concentrerà anche sulla differenza fra il metodo con e senza spelling correction. Il procedimento per creare un clustering nel caso di spelling correction segue quello realizzato nell'altro notebook.

In [2]:
with open('wspell_pattern', "rb") as input_file:
    wspell_pattern = pickle.load(input_file)
    
with open('tspell_pattern', "rb") as input_file:
    tspell_pattern = pickle.load(input_file)

<font size=3>
    
Anche in questo caso si devono identificare i pattern rilevanti. Non è scontato che con la spelling correction alcune parole siano state etichettate in modo differente e quindi anche la quantità e i tipi di sequenze identificate possano differire dall'altro metodo. Di norma i tagger etichettano come nome le parole che non hanno nel loro vocabolario, perché possono essere nomi di persona o di entità non conosciute, quindi è una scelta che la maggior parte delle volte dovrebbe essere corretta. L'assegnazione di nomi a parole scritte erroneamente va a formare un pattern che in realtà potrebbe non essere un pattern reale perché la parte del discorso corretta non è un nome.

Il seguente codice ricrea le liste con i pattern più rilevanti.

In [5]:
def relevant_pattern(pattern_diz, pattern_list, word_list):
    w_gsp = []
    t_gsp = []
    for i in tqdm(range(len(pattern_list))):
        w_gsp_temp = []
        t_gsp_temp = []
        for j in range(len(pattern_list[i])):
            for k in range(len(pattern_list[i][j])):
                if pattern_list[i][j][k] in pattern_diz.keys():
                    w_gsp_temp.append(word_list[i][j][k])
                    t_gsp_temp.append(pattern_list[i][j][k])
                else:
                    w_gsp_temp.append("")
                    t_gsp_temp.append("")
    
        w_gsp.append(w_gsp_temp)
        t_gsp.append(t_gsp_temp)
    
    return(w_gsp, t_gsp)

In [6]:
# con spelling correction
di_p2 = [pattern for aspect in tspell_pattern for sentence in aspect for pattern in sentence if pattern != '']
di_p2 = dict(Counter(di_p2).most_common())
            
pat_temp2 = Counter(di_p2).most_common()

# si seleziona lo 0.5% più comune
gsp2 = Counter(di_p2).most_common(int(len(pat_temp2)/200))

# si passa da tupla a dizionario
gsp2_diz = {}
for pattern in gsp2:
    if pattern[0] not in gsp2_diz.keys():
        gsp2_diz[pattern[0]] = 1
        
# si creano due nuove liste di parole e tag relativi solamente ai pattern rilevanti
wspell_pattern_gsp, tspell_pattern_gsp = relevant_pattern(gsp2_diz, tspell_pattern, wspell_pattern)

100%|███████████████████████████████████████████████████████████████████████| 789780/789780 [00:11<00:00, 68141.77it/s]


In [7]:
def find_word(w_list, w_pattern, w_output, nouns_list):
    for i in tqdm(range(len(w_list))):
        for j in range(len(w_list[i])):
            
            # le parole e i tag sono in un'unica stringa, si separano
            temp_w = w_list[i][j].split(' ') 
            temp_p = w_pattern[i][j].split(' ')
            
            for k in range(len(temp_p)):
                if temp_p[k] == "NN":
                    out = cases(temp_w[k])
                    nouns_list.append(temp_w[k])
                    
                    # Si controlla che non si tratti di un nome di persona
                    if temp_w[k] != temp_w[k].lower():
                        if k < (len(temp_p) - 1):
                            if (temp_w[k-1] != temp_w[k-1].lower()) or (temp_w[k+1] != temp_w[k+1].lower()):
                                break
                        else:
                            if (temp_w[k-1] != temp_w[k-1].lower()):
                                break
                            
                    # Le uniche maiuscole accettate sono quelle iniziali e si controlla se il lemma è un nome
                    if out in ['NU', 'FU']:
                        temp = wordnet.synsets(temp_w[k])
                        if len(temp) > 0:
                            for w in range(len(temp)):
                                word = ''
                                if '.n.' in temp[w].name():
                                    word = temp[w]
                                    break
                                  
                            if temp_w[k].lower() not in w_output.keys():
                                if (temp_w[k] not in w_output.keys()) and (len(temp_w[k]) > 2) and (type(word) != str):
                                    w_output[temp_w[k]] = word
                                    
                                        
    return(w_output)

<font size=3>
    
Si passano le parole, si controllano che siano nomi e riconosciuti da `WordNet`. In caso affermativo sono aggiunti alla liste delle parole che saranno i nuovi punti di riferimento per la matrice.

In [8]:
word_accepted_spell = {}
nouns_list_spell = []
word_spell = find_word(wspell_pattern_gsp, tspell_pattern_gsp, w_output=word_accepted_spell, nouns_list=nouns_list_spell)

100%|████████████████████████████████████████████████████████████████████████| 789780/789780 [01:53<00:00, 6952.00it/s]


In [9]:
word_accepted_spell_l = word_accepted_spell_l2 = [word_accepted_spell[key] for key in word_accepted_spell.keys()]

<font size=3>
    
Si crea un nuova matrice delle distanze.

In [10]:
print(len(word_accepted_spell_l))

m = len(word_accepted_spell_l)
distances_spell = dia_matrix((m , m)).toarray()

20600


<font size=3>
    
Nelle future operazioni sarà necessario modificare la matrice, la lista di parole, che differisce dalla versione senza spelling è salvata in un oggetto e ricaricata per future operazioni.

In [14]:
"""
with open("word_accepted_spell.txt", "w") as w:
    for word in word_accepted_spell.keys():
        w.write(word + '\n')
"""

words_spell = []
with open("word_accepted_spell.txt", "r") as w:
    for word in w:
        words_spell.append(word.strip('\n'))

<font size=3>
    
Ci calcolano le similarità specificano il parametro `simulate_root` su falso in modo tale che per le parole che non hanno una parte dell'albero in comune all'interno di WordNet non si vada a creare un albero fittizio per stimare la similarità, ma in quei casi la similarità sarà nulla.

In [15]:
from nltk.corpus import wordnet as wn
cont = 0
for i in tqdm(range(len(word_accepted_spell_l))):
    j = i
    while j < len(word_accepted_spell_l2):
        distances_spell[i, j] = wn.wup_similarity(word_accepted_spell_l[i], word_accepted_spell_l2[j], simulate_root=False)
        j = j + 1

100%|██████████████████████████████████████████████████████████████████████████| 20600/20600 [5:05:28<00:00,  1.12it/s]


In [16]:
distances_spell = distances_spell.round(3)

<font size=3>
    
La nuova matrice viene salvata e caricata per le future operazioni.

In [17]:
np.save('distances_spell.npy', distances_spell)

In [ ]:
#np.save('distances_spell.npy', distances)

distances_spell = np.load("distances_spell.npy")

<font size=3>
    
Avendo una nuova matrice di similarità è possibile che ci siano più termini con massima similarità da considerare sinonimi, quindi si deve creare un nuovo dizionario e non si può fare affidamento su quello precedente.

In [18]:
diz_similarity_spell = {}
for i in tqdm(range(distances_spell.shape[0])):
    target = words_spell[i]
    sim = []
    for j in range(distances_spell.shape[1]):
        if distances_spell[i, j] == 1 and (i != j):
            sim.append(words_spell[j])
            
    diz_similarity_spell[target] = sim

100%|███████████████████████████████████████████████████████████████████████████| 20600/20600 [02:00<00:00, 170.32it/s]


<font size=3>
    
Si identificano le parole sinonimi che sono importanti ma superflue per la cluster analysis in quanto ci sono già dei termini che possono essere presi al loro posto. Si identificano le posizioni di queste parole all'interno delle liste e si inserisce un elemento nullo.

In [19]:
primary_words = []
filter_words = []
for word in diz_similarity_spell.keys():
    if (word not in primary_words) and (word not in filter_words):
        primary_words.append(word)
    else:
        primary_words.append('')
    
    for w in diz_similarity_spell[word]:
        filter_words.append(w)
        
removed = sum([1 for word in primary_words if word == ''])
print(removed)
removed == len(set(filter_words))

4105


True

<font size=3>
    
La matrice viene ridotta e gli elementi sono rimossi in corrispondenza delle posizioni nelle quali è stato inserito un elemento nullo.

In [20]:
idx = [i for i, x in enumerate(primary_words) if x == '']
idx.reverse()

distances_spell = np.delete(distances_spell, (idx), axis=0)
distances_spell = np.delete(distances_spell, (idx), axis=1)

for i in idx:
    del primary_words[i]

<font size=3>
    
Ottenuta una matrice più piccola, si calcolano i valori della parte triangolare superiore come il reciproco degli indici nella parte superiore in quanto per definizione la similarità è simmetrica. Si arrotondano i valori e si passa ad una vera e propria matrice, fino a questo momento è stato utilizzato un array, e si fa il complementare ad uno per ottenere una matrice delle distanze e non di similarità dato che gli algoritmi di clustering richiedono in input questa tipologia di matrici.

In [21]:
for i in tqdm(range(distances_spell.shape[0])):
    for j in range(distances_spell.shape[1]):
        if j < i:
            distances_spell[i, j] = distances_spell[j, i]

distances_spell = distances_spell.round(3)

distances_spell = np.matrix(distances_spell)
d_matrix_spell = 1 - distances_spell

100%|███████████████████████████████████████████████████████████████████████████| 16495/16495 [00:50<00:00, 327.65it/s]


In [23]:
link = ['average', 'complete']
silhouette_score_spell, CH_score_spell = [[] for i in range(2)]
(silhouette_spell, CH_spell) = clustering(5, 31, link_list=link, dist=d_matrix_spell, 
                              silhouette_score=silhouette_score_spell, CH_score=CH_score_spell)

In [24]:
c = [i for i in range(5, 31)]
h_plot_evaluation(title="[Spell] Silhouette index", metric=silhouette_spell[0], c=c, label1="Average",
                  metric2=silhouette_spell[1], label2="Complete", pos_legend="top_right")

<font size=3>
    
Si prendono i mille termini più frequenti.

In [25]:
common_words = Counter(nouns_list_spell).most_common()
primary_words2 = []
filter_words2 = []
for word in common_words[0:1000]:
    if word[0] in diz_similarity_spell.keys():
        for w in diz_similarity_spell[word[0]]:
            filter_words2.append(w)

    if word[0] not in filter_words2:
        primary_words2.append(word[0])
    else:
        primary_words2.append('')

<font size=3>
    
Si tengono nella matrice solamente gli elementi necessari.

In [27]:
idx2 = [primary_words.index(word) for word in primary_words if word not in primary_words2]
idx2.reverse()

n_matrix_spell = np.delete(d_matrix_spell, (idx2), axis=0)
n_matrix_spell = np.delete(n_matrix_spell, (idx2), axis=1)

for i in idx2:
    del primary_words[i]
    
print(n_matrix_spell.shape)

(852, 852)


<font size=3>
    
Si replicano i modelli di clustering.

In [30]:
# complementare
silhouette_score_spell2, CH_score_spell2 = [[] for i in range(2)]
(silhouette_spell2, CH_spell2) = clustering(2, 31, link_list=link, dist=n_matrix_spell, 
                              silhouette_score=silhouette_score_spell2, CH_score=CH_score_spell2)

In [32]:
c = [i for i in range(2, 31)]
h_plot_evaluation(title="[Spell] Silhouette index", metric=silhouette_spell2[0], c=c, label1="Average",
                  metric2=silhouette_spell2[1], label2="Complete", pos_legend="top_right")

In [35]:
model = AgglomerativeClustering(affinity='precomputed', n_clusters=11, linkage="complete").fit(n_matrix_spell1)
labels = model.labels_
Counter(labels).most_common()

[(0, 153),
 (6, 113),
 (1, 83),
 (5, 80),
 (2, 75),
 (4, 68),
 (3, 64),
 (8, 62),
 (9, 60),
 (7, 53),
 (10, 41)]

In [36]:
temp = pd.DataFrame({'Word': primary_words, 'Cluster': labels})

temp["Freq"] = temp["Word"].map(dict(common_words))
temp = temp.sort_values(by='Freq', ascending=False)

temp.head()
temp.to_csv(path_or_buf="D:/Scraping/labels_spell2.csv", index=False)